## Importing Libraries

In [49]:
!git clone https://github.com/ayushitamboli/phonepe_pulse.git

fatal: destination path 'phonepe_pulse' already exists and is not an empty directory.


In [19]:
import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image

# Data Extraction

### Cloning Data from Github

In [20]:
# os.environ["GIT_PYTHON_REFRESH"] = "quiet"
# import git
# from git.repo.base import Repo
# Repo.clone_from("https://github.com/PhonePe/pulse.git", "C:/Users/SANTHOSH/Project_3_PhonepePulse/data")

# Data transformation

## Dataframe of aggregated Transactions

In [21]:
path="/Users/bannu/Downloads/pulse-master/pulse-master/data/aggregated/transaction/country/india/state"
Agg_state_list=os.listdir(path)

In [22]:
path1 = "/Users/bannu/Downloads/pulse-master/pulse-master/data/aggregated/transaction/country/india/state"
agg_tran_list = sorted([entry for entry in os.listdir(path1) if os.path.isdir(os.path.join(path1, entry))])

columns1 = {"States":[], "Years":[], "Quarter":[], "Transaction_Type":[], "Transaction_Count":[], "Transaction_Amount":[]}
for state in agg_tran_list:
    cur_states = os.path.join(path1, state)

    # Check if the current entry is a directory
    if os.path.isdir(cur_states):
        agg_year_list = [entry for entry in sorted(os.listdir(cur_states)) if os.path.isdir(os.path.join(cur_states, entry))]
    
        for year in agg_year_list:
            # Use quote to handle special characters in the state name and year
            cur_year = os.path.join(cur_states,(year))
            agg_file_list = os.listdir(cur_year)
            
            for file in agg_file_list:
                cur_file = os.path.join(cur_year, file)

                # Check if the current entry is a file
                data = open(cur_file, "r")
                A = json.load(data)
                    
                for i in A["data"]["transactionData"]:
                    name = i["name"]
                    count = i["paymentInstruments"][0]["count"]
                    amount = i["paymentInstruments"][0]["amount"]
                    columns1["Transaction_Type"].append(name)
                    columns1["Transaction_Count"].append(count)
                    columns1["Transaction_Amount"].append(amount)
                    columns1["States"].append(state)
                    columns1["Years"].append(year)
                    columns1["Quarter"].append(int(file.strip(".json")))
                    


In [23]:
aggre_transaction = pd.DataFrame(columns1)

In [24]:
aggre_transaction["States"] = aggre_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
aggre_transaction["States"] = aggre_transaction["States"].str.replace("-"," ")
aggre_transaction["States"] = aggre_transaction["States"].str.title()
aggre_transaction['States'] = aggre_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")



In [25]:
aggre_transaction.shape

(5034, 6)

## Dataframe of aggregated user

In [26]:
path2 = "/Users/bannu/Downloads/pulse-master/pulse-master/data/aggregated/user/country/india/state"
agg_user_list = sorted([entry for entry in os.listdir(path2) if os.path.isdir(os.path.join(path2, entry))])

columns2 = {"States":[], "Years":[], "Quarter":[], "Brands":[], "Count":[], "Percentage":[]}
for state in agg_user_list:
    cur_states = os.path.join(path2, state)

    # Check if the current entry is a directory
    if os.path.isdir(cur_states):
        agg_year_list = [entry for entry in sorted(os.listdir(cur_states)) if os.path.isdir(os.path.join(cur_states, entry))]
    
        for year in agg_year_list:
            # Use quote to handle special characters in the state name and year
            cur_year = os.path.join(cur_states,(year))
            agg_file_list = os.listdir(cur_year)
            
            for file in agg_file_list:
                cur_file = os.path.join(cur_year, file)

                # Check if the current entry is a file
                data = open(cur_file, "r")
                B = json.load(data)
                
                try:
                    for i in B["data"]["usersByDevice"]:
                        brand = i["brand"]
                        count = i["count"]
                        percentage = i["percentage"]
                        columns2["Brands"].append(brand)
                        columns2["Count"].append(count)
                        columns2["Percentage"].append(percentage)
                        columns2["States"].append(state)
                        columns2["Years"].append(year)
                        columns2["Quarter"].append(int(file.strip(".json")))
                except:
                    pass


In [27]:
aggre_user = pd.DataFrame(columns2)

In [28]:
aggre_user

,States,Years,Quarter,Brands,Count,Percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,0.247033
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,0.214392
2,andaman-&-nicobar-islands,2018,1,Vivo,982,0.145697
3,andaman-&-nicobar-islands,2018,1,Oppo,501,0.074332
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,0.015056
6728,west-bengal,2022,1,Infinix,284678,0.012987
6729,west-bengal,2022,1,Asus,280347,0.012790
6730,west-bengal,2022,1,Apple,277752,0.012671


In [29]:
aggre_user["States"] = aggre_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
aggre_user["States"] = aggre_user["States"].str.replace("-"," ")
aggre_user["States"] = aggre_user["States"].str.title()
aggre_user['States'] = aggre_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")


## Dataframe of map transactions

In [30]:
#Map_transaction
path3 = "/Users/bannu/Downloads/pulse-master/pulse-master/data/map/transaction/hover/country/india/state"
map_tran_list = sorted([entry for entry in os.listdir(path3) if os.path.isdir(os.path.join(path3, entry))])

columns3 = {"States":[], "Years":[], "Quarter":[], "Districts":[], "Transaction_Count":[], "Transaction_Amount":[]}
for state in map_tran_list:
    cur_states = os.path.join(path3, state)

    # Check if the current entry is a directory
    if os.path.isdir(cur_states):
        agg_year_list = [entry for entry in sorted(os.listdir(cur_states)) if os.path.isdir(os.path.join(cur_states, entry))]
    
        for year in agg_year_list:
            # Use quote to handle special characters in the state name and year
            cur_year = os.path.join(cur_states,(year))
            agg_file_list = os.listdir(cur_year)
            
            for file in agg_file_list:
                cur_file = os.path.join(cur_year, file)

                # Check if the current entry is a file
                data = open(cur_file, "r")
                C = json.load(data)
                
                for i in C["data"]["hoverDataList"]:
                        name = i["name"]
                        count = i["metric"][0]["count"]
                        amount = i["metric"][0]["amount"]
                        columns3["Districts"].append(name)
                        columns3["Transaction_Count"].append(count)
                        columns3["Transaction_Amount"].append(amount)
                        columns3["States"].append(state)
                        columns3["Years"].append(year)
                        columns3["Quarter"].append(int(file.strip(".json")))
map_tran = pd.DataFrame(columns3)


In [31]:
map_tran

,States,Years,Quarter,Districts,Transaction_Count,Transaction_Amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman district,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars district,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
20599,west-bengal,2024,4,alipurduar district,15875637,2.099251e+10
20600,west-bengal,2024,4,paschim bardhaman district,56616799,6.968735e+10
20601,west-bengal,2024,4,nadia district,65274337,1.079320e+11
20602,west-bengal,2024,4,birbhum district,36905213,5.778701e+10


## Dataframe of map user

In [32]:
path4 ="/Users/bannu/Downloads/pulse-master/pulse-master/data/map/user/hover/country/india/state"



if not os.path.exists(path4):
    print(f"Error: Path does not exist - {path4}")
else:
    map_user_list = os.listdir(path4)

    columns4 = {"State": [], "Year": [], "Quarter": [], "District": [],
                "RegisteredUser": [], "AppOpens": []}

    for state in map_user_list:
        cur_state = os.path.join(path4, state)

        if not os.path.isdir(cur_state):  # Skip if it's not a directory
            continue

        map_year_list = os.listdir(cur_state)

        for year in map_year_list:
            cur_year = os.path.join(cur_state, year)

            if not os.path.isdir(cur_year):
                continue

            map_file_list = os.listdir(cur_year)

            for file in map_file_list:
                cur_file = os.path.join(cur_year, file)

                try:
                    with open(cur_file, 'r', encoding="utf-8") as data:
                        D = json.load(data)
                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(f"Error reading file {cur_file}: {e}")
                    continue

                if "data" not in D or "hoverData" not in D["data"]:
                    print(f"Skipping {cur_file} - 'hoverData' missing")
                    continue

                for district, values in D["data"]["hoverData"].items():
                    if "registeredUsers" in values and "appOpens" in values:
                        columns4["District"].append(district)
                        columns4["RegisteredUser"].append(values["registeredUsers"])
                        columns4["AppOpens"].append(values["appOpens"])
                        columns4['State'].append(state)
                        columns4['Year'].append(year)
                        columns4['Quarter'].append(int(file.strip('.json')))

    df_map_user = pd.DataFrame(columns4)


In [33]:
df_map_user

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
20603,west-bengal,2024,4,alipurduar district,475688,31842355
20604,west-bengal,2024,4,paschim bardhaman district,1468252,80543469
20605,west-bengal,2024,4,nadia district,1861738,98740305
20606,west-bengal,2024,4,birbhum district,1114220,73465525


## Dataframe of top transactions

In [34]:
path5 = "/Users/bannu/Downloads/pulse-master/pulse-master/data/top/transaction/country/india/state"
top_tran_list = sorted([entry for entry in os.listdir(path5) if os.path.isdir(os.path.join(path5, entry))])

columns5 = {"States":[], "Years":[], "Quarter":[], "Pincodes":[], "Transaction_count":[], "Transaction_amount":[]}
for state in top_tran_list:
    cur_states = os.path.join(path5, state)

    # Check if the current entry is a directory
    if os.path.isdir(cur_states):
        agg_year_list = [entry for entry in sorted(os.listdir(cur_states)) if os.path.isdir(os.path.join(cur_states, entry))]
    
        for year in agg_year_list:
            # Use quote to handle special characters in the state name and year
            cur_year = os.path.join(cur_states,(year))
            agg_file_list = os.listdir(cur_year)
            
            for file in agg_file_list:
                cur_file = os.path.join(cur_year, file)

                # Check if the current entry is a file
                data = open(cur_file, "r")
                E = json.load(data)
                
                for i in E["data"]["pincodes"]:
                    entityname = i["entityName"]
                    count = i["metric"]["count"]
                    amount = i["metric"]["amount"]
                    columns5["Pincodes"].append(entityname)
                    columns5["Transaction_count"].append(count)
                    columns5["Transaction_amount"].append(amount)
                    columns5["States"].append(state)
                    columns5["Years"].append(year)
                    columns5["Quarter"].append(int(file.strip(".json")))

top_transaction = pd.DataFrame(columns5)

top_transaction["States"] = top_transaction["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_transaction["States"] = top_transaction["States"].str.replace("-"," ")
top_transaction["States"] = top_transaction["States"].str.title()
top_transaction['States'] = top_transaction['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")



In [35]:
top_transaction


,States,Years,Quarter,Pincodes,Transaction_count,Transaction_amount
0,Andaman & Nicobar,2018,1,744101,1622,2.769298e+06
1,Andaman & Nicobar,2018,1,744103,1223,2.238042e+06
2,Andaman & Nicobar,2018,1,744102,969,3.519060e+06
3,Andaman & Nicobar,2018,1,744105,685,1.298561e+06
4,Andaman & Nicobar,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
9994,West Bengal,2024,4,711101,6753348,1.049483e+10
9995,West Bengal,2024,4,700059,6662715,8.009476e+09
9996,West Bengal,2024,4,700039,6575693,9.018748e+09
9997,West Bengal,2024,4,734001,6432082,8.996529e+09


## Dataframe of top users

In [36]:
path6 = "/Users/bannu/Downloads/pulse-master/pulse-master/data/top/user/country/india/state"
top_user_list = sorted([entry for entry in os.listdir(path6) if os.path.isdir(os.path.join(path6, entry))])

columns6 = {"States":[], "Years":[], "Quarter":[], "Pincodes":[], "RegisteredUser":[]}
for state in top_user_list:
    cur_states = os.path.join(path6, state)

    # Check if the current entry is a directory
    if os.path.isdir(cur_states):
        agg_year_list = [entry for entry in sorted(os.listdir(cur_states)) if os.path.isdir(os.path.join(cur_states, entry))]
    
        for year in agg_year_list:
            # Use quote to handle special characters in the state name and year
            cur_year = os.path.join(cur_states,(year))
            agg_file_list = os.listdir(cur_year)
            
            for file in agg_file_list:
                cur_file = os.path.join(cur_year, file)

                # Check if the current entry is a file
                data = open(cur_file, "r")
                F = json.load(data)
                
                for i in F["data"]["pincodes"]:
                    name = i["name"]
                    registeredusers = i["registeredUsers"]
                    columns6["Pincodes"].append(name)
                    columns6["RegisteredUser"].append(registeredusers)
                    columns6["States"].append(state)
                    columns6["Years"].append(year)
                    columns6["Quarter"].append(int(file.strip(".json")))
top_user = pd.DataFrame(columns6)

top_user["States"] = top_user["States"].str.replace("andaman-&-nicobar-islands","Andaman & Nicobar")
top_user["States"] = top_user["States"].str.replace("-"," ")
top_user["States"] = top_user["States"].str.title()
top_user['States'] = top_user['States'].str.replace("Dadra & Nagar Haveli & Daman & Diu", "Dadra and Nagar Haveli and Daman and Diu")




In [37]:
top_user

,States,Years,Quarter,Pincodes,RegisteredUser
0,Andaman & Nicobar,2018,1,744103,1608
1,Andaman & Nicobar,2018,1,744101,1108
2,Andaman & Nicobar,2018,1,744105,1075
3,Andaman & Nicobar,2018,1,744102,1006
4,Andaman & Nicobar,2018,1,744104,272
...,...,...,...,...,...
9995,West Bengal,2024,4,733134,169596
9996,West Bengal,2024,4,700059,169448
9997,West Bengal,2024,4,711302,146034
9998,West Bengal,2024,4,700150,142151


## Converting Dataframes to CSV files

In [38]:
aggre_transaction.to_csv('aggre_transaction.csv',index = False)
aggre_user.to_csv('aggre_user.csv',index = False)
map_tran.to_csv('map_tran.csv',index = False)
df_map_user.to_csv('df_map_user.csv',index = False)
top_transaction.to_csv('top_transaction.csv',index = False)
top_user.to_csv('top_user.csv',index = False)


# Creating connection with MySQL

### Connecting with SQL

In [55]:
mydb = sql.connect(host="localhost",
                   user="root",
                   password="",
                   database= "phonepe_pulse"
                  )
mycursor = mydb.cursor(buffered=True)

### Creating agg_trans table

In [ ]:
mycursor.execute("create table aggre_transaction (State varchar(100), Year int, Quarter int, Transaction_type varchar(100), Transaction_count int, Transaction_amount double)")

for i,row in aggre_transaction.iterrows():
    #here %S means string values 
    sql = "INSERT INTO aggre_transaction VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    mydb.commit()

### Creating agg_user table

In [ ]:
mycursor.execute("create table aggre_user (State varchar(100), Year int, Quarter int, Brands varchar(100), Count int, Percentage double)")

for i,row in aggre_user.iterrows():
    sql = "INSERT INTO aggre_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

### Creating map_trans table

In [ ]:
mycursor.execute("create table map_trans (State varchar(100), Year int, Quarter int, District varchar(100), Count int, Amount double)")

for i,row in map_tran.iterrows():
    sql = "INSERT INTO map_trans VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

### Creating map_user table

In [ ]:
mycursor.execute("create table map_user (State varchar(100), Year int, Quarter int, District varchar(100), Registered_user int, App_opens int)")

for i,row in df_map_user.iterrows():
    sql = "INSERT INTO map_user VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

### Creating top_trans table

In [ ]:
mycursor.execute("create table top_trans (State varchar(100), Year int, Quarter int, Pincode int, Transaction_count int, Transaction_amount double)")

for i,row in top_transaction.iterrows():
    sql = "INSERT INTO top_tran VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

### Creating top_user table

In [ ]:
mycursor.execute("create table top_user (State varchar(100), Year int, Quarter int, Pincode int, Registered_users int)")

for i,row in df_top_user.iterrows():
    sql = "INSERT INTO top_user VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

### List of tables

In [42]:
mycursor.execute("show tables")
mycursor.fetchall()

[('agg_trans',),
 ('agg_user',),
 ('aggre_transaction',),
 ('aggre_user',),
 ('map_trans',),
 ('map_transaction',),
 ('map_user',),
 ('top_trans',),
 ('top_transaction',),
 ('top_user',)]